# Shadow models audit for a PyTorch model trained on Purchase100

## Introduction

In this tutorial, we will see:
* How to create a Dataset object manually
* How to audit a PyTorch model
* How to use the ShadowMetric

## Imports

In [2]:
from math import ceil
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from torch import nn, optim, Tensor

For now we install the library from the local source. A version will be pushed to pip soon.

In [3]:
import sys
!{sys.executable} -m pip install -e ../.
from privacy_meter.audit import Audit
from privacy_meter.dataset import Dataset
from privacy_meter.hypothesis_test import threshold_func
from privacy_meter.information_source import InformationSource
from privacy_meter.information_source_signal import ModelLoss
from privacy_meter.metric import ShadowMetric
from privacy_meter.model import PytorchModel

Obtaining file:///home/victor/ml_privacy_meter
  Preparing metadata (setup.py) ... done
  Attempting uninstall: privacy-meter
    Found existing installation: privacy-meter 1.0
    Uninstalling privacy-meter-1.0:
      Successfully uninstalled privacy-meter-1.0
  Running setup.py develop for privacy-meter


2022-03-28 11:44:49.984452: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 11:44:49.984478: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Settings

Setting seed for reproducibility:

In [4]:
np.random.seed(1234)

Hyper parameters:

In [5]:
n_shadow_models = 3  # Number of shadow models to be trained
epochs = 10          # Number of epochs for each model
batch_size = 8       # Batch size for the trainings

## Dataset creation

Let's download the Purchase100 dataset (presented in https://www.cs.cornell.edu/~shmat/shmat_oak17.pdf on page 7) and extract it:

In [6]:
!wget https://github.com/privacytrustlab/datasets/raw/master/dataset_purchase.tgz
!tar -xvzf dataset_purchase.tgz
!rm dataset_purchase.tgz

--2022-03-28 11:44:52--  https://github.com/privacytrustlab/datasets/raw/master/dataset_purchase.tgz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/privacytrustlab/datasets/master/dataset_purchase.tgz [following]
--2022-03-28 11:44:52--  https://raw.githubusercontent.com/privacytrustlab/datasets/master/dataset_purchase.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22045876 (21M) [application/octet-stream]
Saving to: ‘dataset_purchase.tgz’

dataset_purchase.tg 100%[===================>]  21.02M  11.1MB/s    in 1.9s    

2022-03-28 11:44:58 (11.1 MB/s) - ‘dataset_purchase.tgz’ saved [22

We need to read the file and preprocess the dataset:

In [7]:
def preprocess_purchase100():
    """
    Cf. https://www.cs.cornell.edu/~shmat/shmat_oak17.pdf page 7
    Returns:

    """
    # Read raw dataset
    dataset_path = "dataset_purchase"
    with open(dataset_path, "r") as f:
        purchase_dataset = f.readlines()
    # Separate features and labels into different arrays
    x, y = [], []
    for datapoint in purchase_dataset:
        split = datapoint.rstrip().split(",")
        label = int(split[0]) - 1  # The first value is the label
        features = np.array(split[1:], dtype=np.float32)  # The next values are the features
        x.append(features)
        y.append(label)
    # Make sure the datatype is correct
    x = np.array(x, dtype=np.float32)
    # Convert labels into one hot vectors
    y = OneHotEncoder(sparse=False).fit_transform(np.expand_dims(y, axis=1))
    # Split data into train, test sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1234)
    return x_train, y_train, x_test, y_test

In [8]:
x_train, y_train, x_test, y_test = preprocess_purchase100()

Then, we wrap the dataset into a Dataset object:
* `data_dict` contains the actual dataset, in the form of a 2D dictionary. The first key corresponds to the split name (here we have two: "train" and "test"), and the second key to the feature name (here we also have two: "x" and "y").
* `default_input` contains the name of the feature that should be used as the models input (here "x").
* `default_output` contains the name of the feature that should be used as the label / models output (here "y").

In [9]:
dataset = Dataset(
    data_dict={'train': {'x': x_train, 'y': y_train}, 'test': {'x': x_test, 'y': y_test}},
    default_input='x',
    default_output='y'
)

Finally, we use the built-in `Dataset.subdivide()` function, to split the two splits ("train" and "test") into multiple sub-splits (one per model). Their names will be "train000", "train001", etc. and "test000", "test001", etc.

In [10]:
dataset.subdivide(num_splits=n_shadow_models + 1, delete_original=True)

Let's now define the pytorch models to be used: one target model, and `n_shadow_models` shadow models.

In [11]:
torch_models = [
    nn.Sequential(
        nn.Linear(in_features=600, out_features=128),
        nn.Tanh(),
        nn.Linear(in_features=128, out_features=100),
        nn.Softmax(dim=1)
    )
    for _ in range(n_shadow_models + 1)
]

We define the loss:

In [12]:
criterion = nn.CrossEntropyLoss(reduction='sum')

And we train each model on its split of the dataset:

In [13]:
for k, model in enumerate(torch_models):
    optimizer = optim.Adam(model.parameters())
    x = dataset.get_feature(split_name=f'train{k:03d}', feature_name='<default_input>')
    y = dataset.get_feature(split_name=f'train{k:03d}', feature_name='<default_output>')
    n_samples = x.shape[0]
    n_batches = ceil(n_samples / batch_size)
    x = np.array_split(x, n_batches)
    y = np.array_split(y, n_batches)
    for epoch in range(epochs):
        epoch_loss, acc = 0.0, 0.0
        for b in range(n_batches):
            optimizer.zero_grad()
            y_pred = model(Tensor(x[b]))
            loss = criterion(Tensor(y[b]), y_pred)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            acc += torch.sum(y_pred.argmax(axis=1) == Tensor(y[b]).argmax(axis=1))
        acc /= n_samples
        epoch_loss /= n_samples
        print(f'model #{k:02d}, epoch #{epoch:03d}:\ttrain_acc = {acc:.3f}\ttrain_loss = {epoch_loss:.3e}')

/home/victor/ml_privacy_meter/venv/lib/python3.8/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


model #00, epoch #000:	train_acc = 0.252	train_loss = 4.394e+00
model #00, epoch #001:	train_acc = 0.394	train_loss = 4.250e+00
model #00, epoch #002:	train_acc = 0.455	train_loss = 4.188e+00
model #00, epoch #003:	train_acc = 0.512	train_loss = 4.131e+00
model #00, epoch #004:	train_acc = 0.537	train_loss = 4.102e+00
model #00, epoch #005:	train_acc = 0.559	train_loss = 4.078e+00
model #00, epoch #006:	train_acc = 0.580	train_loss = 4.056e+00
model #00, epoch #007:	train_acc = 0.612	train_loss = 4.025e+00
model #00, epoch #008:	train_acc = 0.626	train_loss = 4.008e+00
model #00, epoch #009:	train_acc = 0.633	train_loss = 3.999e+00
model #01, epoch #000:	train_acc = 0.271	train_loss = 4.377e+00
model #01, epoch #001:	train_acc = 0.411	train_loss = 4.235e+00
model #01, epoch #002:	train_acc = 0.460	train_loss = 4.182e+00
model #01, epoch #003:	train_acc = 0.498	train_loss = 4.141e+00
model #01, epoch #004:	train_acc = 0.529	train_loss = 4.109e+00
model #01, epoch #005:	train_acc = 0.573

Now that the models are all train, we can wrap each of them in a `PytorchModel` object:

In [14]:
models = [
    PytorchModel(
        model_obj=model,
        loss_fn=criterion
    )
    for model in torch_models
]

## Information Sources

We can now define two InformationSource objects. Basically, an information source is an abstraction representing a set of models, and their corresponding dataset. Note that we use the same `dataset` variable for the two InformationSource objects, but each will be queried on different splits of the dataset.

In [15]:
target_info_source = InformationSource(
    models=[models[0]],
    datasets=[dataset]
)

reference_info_source = InformationSource(
    models=models[1:],
    datasets=[dataset]
)

## Metric and Audit

We now create a Metric object, which is an abstraction representing an algorithm used to measure something on an InformationSource, such as membership information leakage. In this case, we use the ShadowMetric to measure the membership information leakage of `target_info_source` in a black-box setting, using shadow models.

In [16]:
metric = ShadowMetric(
    target_info_source=target_info_source,
    reference_info_source=reference_info_source,
    signals=[ModelLoss()],
    hypothesis_test_func=threshold_func,
    unique_dataset=True
)

Finally, the Audit object is a wrapper to actually run the audit, and display the results. More visualization options will be added soon.

In [17]:
audit = Audit(
    metric=metric,
    target_info_source=target_info_source,
    reference_info_source=reference_info_source
)

print(audit.run())

============= METRIC RESULT OBJECT =============
Accuracy          = 0.75
ROC AUC Score     = 0.5
FPR               = 1.0
TN, FP, FN, TP    = (0, 12333, 0, 36999)
